In [9]:
import pandas as pd
import numpy as np
import os 

In [10]:
df = pd.read_csv("BBC_df.csv", encoding = "ISO-8859-1")
df.head()

,article,summary,topic
0,Musicians to tackle US red tape\n\nMusicians' ...,Nigel McCune from the Musicians' Union said Br...,NaN
1,"U2's desire to be number one\n\nU2, who have w...",But they still want more.They have to want to ...,entertainment
2,Rocker Doherty in on-stage fight\n\nRock singe...,"Babyshambles, which he formed after his acrimo...",NaN
3,Snicket tops US box office chart\n\nThe film a...,A Series of Unfortunate Events also stars Scot...,NaN
4,Ocean's Twelve raids box office\n\nOcean's Twe...,"Ocean's Twelve, the crime caper sequel starrin...",entertainment


In [11]:
import langdetect

def detect_language(text):
    return langdetect.detect(text)

df = pd.read_csv("BBC_df.csv", encoding = "ISO-8859-1")
print("Nombre d'inputs avant suppression des textes non anglais : " + str(len(df)))

# créer une liste vide qui contiendra les indices des lignes à supprimer
indexes_to_delete = []

# parcourir chaque ligne du dataframe
for index, row in df.iterrows():

    lang = detect_language(row['article'][0:200])
    # si la valeur de l'attribut 'text' commence par un 'm'
    if type(row["article"]) == float:
    # ajouter l'index de cette ligne à la liste des indices à supprimer
        indexes_to_delete.append(index)

# supprimer les lignes du dataframe en utilisant la liste des indices à supprimer
df.drop(indexes_to_delete, inplace=True)

print("Nombre d'inputs après suppression des textes non anglais : " + str(len(df)))

Nombre d'inputs avant suppression des textes non anglais : 2225
Nombre d'inputs après suppression des textes non anglais : 2225


### PREPROCESSING


In [12]:
# REMOVE PUNCTUATION
import string

def remove_punctuation(summary):
    punctuationfree="".join([i for i in summary if i not in string.punctuation])
    return punctuationfree

df["article"] = df["article"].apply(lambda row : remove_punctuation(row))
df["article"] = df["article"].apply(lambda row: row.lower())

df["summary"] = df["summary"].apply(lambda row : remove_punctuation(row))
df["summary"] = df["summary"].apply(lambda row: row.lower())

In [13]:
# TOKENIZATION
import nltk
import nltk.corpus
from nltk.tokenize import word_tokenize

df["article"] = df["article"].apply(lambda x : word_tokenize(x))

df["summary"] = df["summary"].apply(lambda x : word_tokenize(x))

In [14]:
# REMOVE STOP WORDS
nltk.download('stopwords')
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
df["article"] = df["article"].apply(lambda x: [w for w in x if not w in stop_words])

df["summary"] = df["summary"].apply(lambda x: [w for w in x if not w in stop_words])

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/leolamoureux/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
# LEMMATIZATION
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

df["article"] = df["article"].apply(lambda x: [WordNetLemmatizer().lemmatize(w) for w in x])

df["summary"] = df["summary"].apply(lambda x: [WordNetLemmatizer().lemmatize(w) for w in x])

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/leolamoureux/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [16]:
df.to_csv('preprocessed_df.csv', index=False)